# Hanle Effect Analysis

In [2]:
# Run this cell with Shift-Enter, and wait until the 
# asterisk changes to a number, i.e., [*] becomes [1]
import numpy as np
import scipy.constants as const
import uncertainties as unc
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

## Calibration


In [8]:
#Read in data
current_data = pd.read_csv("sweep current data.csv")

#Finding the average minimum and maximum current
min_current = unc.ufloat(np.mean(current_data["Min(A)"]), np.std(current_data["Min(A)"]))
max_current = unc.ufloat(np.mean(current_data["Max(A)"]), np.std(current_data["Max(A)"]))

print("Average minimum current: {:.5f}".format(min_current))
print("Average maximum current: {:.5f}".format(max_current))
current_data

Average minimum current: -0.19755+/-0.00067
Average maximum current: 0.20661+/-0.00181


,Min(A),Max(A)
0,-0.19679,0.20821
1,-0.19876,0.20843
2,-0.19708,0.20978
3,-0.19786,0.20839
4,-0.19744,0.20521
5,-0.19685,0.20515
6,-0.19744,0.20431
7,-0.19678,0.20471
8,-0.19826,0.20596
9,-0.19826,0.20596


In [9]:
#Defining relevant values

#Dwell time per channel
dwell_time = 0.020 #ms
period = 21 #s
#Factor for converting form current to magnetic field
conv_factor = 14.3 #gauss/ampere

#Half width half maximum
hwhm1 = unc.ufloat(29.435198, 3.295223e-5)
hwhm2 = unc.ufloat(28.223673, 3.395952e-5)

In [21]:
#Convert channel number to current and then to magnetic field.

def conv_to_current(hwhm, period, dwell=dwell_time, MinCurrent=min_current, MaxCurrent=max_current):
    
    num_of_bin = period/(2*dwell)
    current_width = (abs(max_current - min_current)) / num_of_bin
    hwhm_current = current_width * hwhm
    return hwhm_current

#Apply conversion function
hwhm_current1 = conv_to_current(hwhm1, period)
hwhm_current2 = conv_to_current(hwhm2, period)

print("The hwhm of peak 1 in units of current: {:.5f} (A)".format(hwhm_current1))
print("The hwhm of peak 2 in units of current: {:.5f} (A)".format(hwhm_current2))

The hwhm of peak 1 in units of current: 0.02266+/-0.00011 (A)
The hwhm of peak 2 in units of current: 0.02173+/-0.00010 (A)


In [22]:
#Converting to magnetic field

#Apply conversion factor
hwhm_mag1 = hwhm_current1*conv_factor
hwhm_mag2 = hwhm_current2*conv_factor

print("The hwhm of peak 1 in units of magnetic field: {:.5f} (gauss)".format(hwhm_mag1))
print("The hwhm of peak 2 in units of magnetic field: {:.5f} (gauss)".format(hwhm_mag2))

The hwhm of peak 1 in units of magnetic field: 0.32404+/-0.00155 (gauss)
The hwhm of peak 2 in units of magnetic field: 0.31070+/-0.00149 (gauss)


## Lifetime

In [29]:
#Find the mean lifetime using both of the vertical polarization peaks.

#Find g_factor

def g_factor(s, l, j):
    g = 1 + ((j*(j + 1) + s*(s+1) - l*(l+1))/(2*j*(j+1)))
    return g

gj = g_factor(1, 1, 1)
#Bohr magneton
mu_B = (const.e*const.hbar)/(2.0*const.m_e)

#Define function for lifetime
def lifetime(hwhm_B, bohr=mu_B, g_J = gj):
    inverse_lt = (2*g_J) * (bohr/const.hbar) * abs(hwhm_B*10**-4)
    lt = 1/inverse_lt

    return lt

lifetime1 = lifetime(hwhm_mag1)
lifetime2 = lifetime(hwhm_mag2)

print("The lifetime of the excited state on the up sweep is: {:.5P} (seconds)".format(lifetime1))
print("The lifetime of the excited state on the down sweep is: {:.5P} (seconds)".format(lifetime2))

The lifetime of the excited state on the up sweep is: (1.1697±0.0056)×10⁻⁷ (seconds)
The lifetime of the excited state on the down sweep is: (1.2199±0.0058)×10⁻⁷ (seconds)
